In [2]:
import os
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

def get_latest_emails(service, max_results=20):
    results = service.users().messages().list(userId='me', maxResults=max_results).execute()
    messages = results.get('messages', [])

    emails = []
    for message in messages:
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        emails.append(msg)

    return emails

# Get Gmail service
gmail_service = get_gmail_service()

# Get latest 20 emails
latest_emails = get_latest_emails(gmail_service)

print(f"Downloaded {len(latest_emails)} emails.")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=796509632999-6mb3rgd6s572gi92vuu8vup6mb5trphu.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60299%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=0wj9h7uyHb0sahNBqjRPjB6uorLRmG&access_type=offline
Downloaded 20 emails.


In [3]:
# Function to extract subject from email
def get_email_subject(email):
    headers = email['payload']['headers']
    subject = next((header['value'] for header in headers if header['name'].lower() == 'subject'), 'No Subject')
    return subject

# Extract and print subjects of the downloaded emails
print("Subjects of the downloaded emails:")
for i, email in enumerate(latest_emails, 1):
    subject = get_email_subject(email)
    print(f"{i}. {subject}")


Subjects of the downloaded emails:
1. Hey, how many C's are there in JavaScript?
2. Here be Ruby dragons
3. New iOS, new macOS, so there's a new Safari, too.
4. The heaviest npm packages
5. An even faster future for JRuby
6. 'Release notes' for the entire web platform?
7. Real-time Ruby profiling
8. Reverse engineering minified JS with ChatGPT
9. Remembering the first decade of web design
10. Security alert
11. Security alert
12. Security alert
13. RubyKaigi direct to your living room
14. JavaScript's Rust tool belt
15. Making your page faster before it even loads
16. A regular expression refresher
17. Thoughts about static typing in Ruby
18. The npm tea party
19. Rails 7.2 released
20. Should you 'over-engineer' your CSS?


In [14]:
from llama_index.core import Document, VectorStoreIndex

# build documents from latest_emails
documents = [Document(text=email.get('snippet')) for email in latest_emails]

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

response = query_engine.query("Tell me about compiling and running C in JavaScript")

print(response)

Compiling and running C in JavaScript involves using tools like Emscripten, which is a source-to-source compiler that can convert C and C++ code into JavaScript. This allows C code to be executed in a web browser or a Node.js environment.
